## Data Cleaning

This notebook has the purpose of: 
- Cleaning and to combine all the reviews into one table.
- Create a wine table, extracted from the the reviews. one wine can be reviewed many times and we need a wine list for each vineyard.

In [1]:
import os
import re
import numpy as np
import pandas as pd
from collections import Counter

raw_spectator_file = "combined_spectator_reviews.csv"
raw_enthusiast_file = "raw_wine_enthusiast_reviews.csv"


In [2]:
spectator_df = pd.read_csv(raw_spectator_file)
spectator_df

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,379875.0,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,429948.0,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,357733.0,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,379943.0,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,380170.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673.0,NaN
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785.0,NaN
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274.0,NaN
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448.0,NaN


In [6]:
enthusiast_df = pd.read_csv(raw_enthusiast_file)
enthusiast_df

,Unnamed: 0,price,rating,bottle_name,region,review_link,source,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,Date Published,User Avg Rating,Importer
0,0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Oregon Other,https://www.winemag.com/buying-guide/the-great...,Wine Enthusiast,"['$13', 'Buy Now']",['Rascal'],['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['The Great Oregon Wine Company'],['13.3%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
1,1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Oregon Other,https://www.winemag.com/buying-guide/wine-by-j...,Wine Enthusiast,"['$19', 'Buy Now']",NaN,['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Wine by Joe'],['13.5%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
2,2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Oregon Other,https://www.winemag.com/buying-guide/duck-pond...,Wine Enthusiast,"['$15', 'Buy Now']",['Oregon Wine Co.'],['Chardonnay'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Duck Pond Cellars'],['13.2%'],['750 ml'],['White'],['9/1/2022'],['Not rated yet [Add Your Review]'],NaN
3,3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,"['$75', 'Buy Now']",['Laurene'],['Pinot Noir'],"['Dundee Hills', ' Willamette Valley', ' Orego...",['Domaine Drouhin'],['13.9%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
4,4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Willamette Valley,https://www.winemag.com/buying-guide/portlandi...,Wine Enthusiast,"['$40', 'Buy Now']",['Momtazi Vineyard'],['Pinot Noir'],"['Willamette Valley', ' Willamette Valley', ' ...",['Portlandia'],['13.5%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,736,$18,90,Ponzi 1998 Arneis (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/ponzi-199...,Wine Enthusiast,"['$18', 'Buy Now']",NaN,"['Arneis', ' Italian White']","['Willamette Valley', ' Willamette Valley', ' ...",['Ponzi'],['N/A'],['500 ml'],['White'],['4/1/2000'],['Not rated yet [Add Your Review]'],NaN
737,737,$12,86,Tyee 1996 Pinot Blanc (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/tyee-1996...,Wine Enthusiast,"['$12', 'Buy Now']",NaN,['Pinot Blanc'],"['Willamette Valley', ' Willamette Valley', ' ...",['Tyee'],['13%'],['750 ml'],['White'],['12/31/1999'],['Not rated yet [Add Your Review]'],NaN
738,738,$35,92,Cameron 1997 Clos Electrique Chardonnay (Willa...,Willamette Valley,https://www.winemag.com/buying-guide/cameron-1...,Wine Enthusiast,"['$35', 'Buy Now']",['Clos Electrique'],['Chardonnay'],"['Willamette Valley', ' Willamette Valley', ' ...",['Cameron'],['12.7%'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN
739,739,$15,90,Torii Mor 1998 Pinot Gris (Yamhill County),Willamette Valley,https://www.winemag.com/buying-guide/torii-mor...,Wine Enthusiast,"['$15', 'Buy Now']",NaN,['Pinot Gris'],"['Yamhill County', ' Willamette Valley', ' Ore...",['Torii Mor'],['N/A'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN


In [3]:
spectator_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       865 non-null    object 
 11  wine_id        865 non-null    float64
 12  Region         373 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 125.9+ KB


In [4]:
spectator_df["winery"].value_counts()

Domaine Serene       57
Bergström            56
Evening Land         45
Beaux Frères         41
Alexana              38
                     ..
Coelho                1
Ancient Cellars       1
Anam Cara Cellars     1
Et Fille              1
Authentique           1
Name: winery, Length: 196, dtype: int64

In [7]:
enthusiast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       741 non-null    int64 
 1   price            739 non-null    object
 2   rating           741 non-null    int64 
 3   bottle_name      741 non-null    object
 4   region           741 non-null    object
 5   review_link      741 non-null    object
 6   source           741 non-null    object
 7   Price            741 non-null    object
 8   Designation      427 non-null    object
 9   Variety          741 non-null    object
 10  Appellation      741 non-null    object
 11  Winery           741 non-null    object
 12  Alcohol          741 non-null    object
 13  Bottle Size      741 non-null    object
 14  Category         741 non-null    object
 15  Date Published   741 non-null    object
 16  User Avg Rating  741 non-null    object
 17  Importer         4 non-null      ob

Starting with the Wine Enthusiast reviews. They seem to be the dirtiest so far

In [24]:
## First task is to get rid of the list appearance of the string formatted data
cleaning_df = enthusiast_df.drop(['Price', "Unnamed: 0", "Importer"], axis=1)
alcohol = cleaning_df['Alcohol'].copy()
alcohol

0      ['13.3%']
1      ['13.5%']
2      ['13.2%']
3      ['13.9%']
4      ['13.5%']
         ...    
736      ['N/A']
737      ['13%']
738    ['12.7%']
739      ['N/A']
740    ['13.2%']
Name: Alcohol, Length: 741, dtype: object

In [25]:
for i in alcohol.index:
    save = re.search(r"\d+\.*\d*%", alcohol[i])
    if save != None:
        alcohol[i] = save.group()
    else:
        alcohol[i] = None
alcohol

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\4171088931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alcohol[i] = save.group()
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\4171088931.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alcohol[i] = None


0      13.3%
1      13.5%
2      13.2%
3      13.9%
4      13.5%
       ...  
736     None
737      13%
738    12.7%
739     None
740    13.2%
Name: Alcohol, Length: 741, dtype: object

In [26]:
cleaning_df['Alcohol'] = alcohol
size = cleaning_df['Bottle Size'].copy()
size

0      ['750 ml']
1      ['750 ml']
2      ['750 ml']
3      ['750 ml']
4      ['750 ml']
          ...    
736    ['500 ml']
737    ['750 ml']
738    ['750 ml']
739    ['750 ml']
740    ['750 ml']
Name: Bottle Size, Length: 741, dtype: object

In [55]:
for i in size.index:
    keep = re.search(r"(\d+\s*ml)", size[i])
    if keep != None:
        pass
        size[i] = keep.group()
    else:
        keep = re.search(r"(\d+\s*ML)", size[i])
        if keep != None:
            pass
            size[i] = keep.group()
        else: 
            keep = re.search(r"(\d+\.\d*\s*L)", size[i])
            size[i] = keep.group()

size

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\2907954254.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size[i] = keep.group()
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\2907954254.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size[i] = keep.group()
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\2907954254.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size[i] = keep.group()


0      750 ml
1      750 ml
2      750 ml
3      750 ml
4      750 ml
        ...  
736    500 ml
737    750 ml
738    750 ml
739    750 ml
740    750 ml
Name: Bottle Size, Length: 741, dtype: object

In [57]:
cleaning_df['Bottle Size'] = size
winery = cleaning_df['Winery'].copy()
winery

0      ['The Great Oregon Wine Company']
1                        ['Wine by Joe']
2                  ['Duck Pond Cellars']
3                    ['Domaine Drouhin']
4                         ['Portlandia']
                     ...                
736                            ['Ponzi']
737                             ['Tyee']
738                          ['Cameron']
739                        ['Torii Mor']
740               ['WillaKenzie Estate']
Name: Winery, Length: 741, dtype: object

In [60]:
for i in winery.index:
    winery[i] = winery[i][2:-2]
    
winery    

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\98839780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winery[i] = winery[i][2:-2]


0      The Great Oregon Wine Company
1                        Wine by Joe
2                  Duck Pond Cellars
3                    Domaine Drouhin
4                         Portlandia
                   ...              
736                            Ponzi
737                             Tyee
738                          Cameron
739                        Torii Mor
740               WillaKenzie Estate
Name: Winery, Length: 741, dtype: object

In [61]:
cleaning_df["Winery"] = winery
variety = cleaning_df["Variety"].copy()
variety

0                    ['Pinot Noir']
1                    ['Pinot Noir']
2                    ['Chardonnay']
3                    ['Pinot Noir']
4                    ['Pinot Noir']
                   ...             
736    ['Arneis', ' Italian White']
737                 ['Pinot Blanc']
738                  ['Chardonnay']
739                  ['Pinot Gris']
740                ['Pinot Grigio']
Name: Variety, Length: 741, dtype: object

In [160]:
# variety = cleaning_df["Variety"]
category = ['Red','White','Sparkling','Rose','Dessert']
##    'Rosé' is a Wine category not a grape. After viewing the reviews by following the review_links
##    the following changes will be made (index, new designation) :  (16, Blend), (80, Pinot Noir), (175, Pinot Noir),
##    (232, Blend), (239, Blend), (240, Blend), (241, Blend), (253, Grenache), (309, Pinot Noir), (310, Blend), 
##    (486, Blend), (602, Sangiovese)

## some wines are made to mathc a specific style, Enthusiast puts that info in the varietal section. 
## It needs to be its own column, save the information

style = []  

for i in variety.index:
    grape = variety[i][2:-2].split("', '")
    
    
    # This column is about grape varietals, mark them as "Blend" if anything that should be a blend. 
    # Save any info about styles
    
    if len(grape) > 1:
        if ("Italian" in grape[1]) or ("Spanish" in grape[1]):
            style.append((i, grape[1]))
            grape = grape[0]
        elif ("Bordeaux" in grape[0]) or ("Bordeaux" in grape[1]) or ("Sparkling" in grape[1]):
            style.append((i,grape))
            grape = "Blend"            
        elif "Other" in grape[1]:
            grape = grape[0]
        elif ("Gamay" in grape[0]) or ("Sémillon" in grape[0]):
            grape = grape[1]
        else: 
            grape = "Blend"
    else:         
        if (i in [16,28,207,232,239,240,241,242,310,486]) or ('Red Blends' in grape[0]) or ("-" in grape[0]):
            grape = "Blend"        
        elif i in [80,175,247,309]: 
            grape = "Pinot Noir"
        elif i == 253:  
            grape = "Grenache"
        elif i == 602: 
            grape= "Sangiovese"
        else: 
            grape = grape[0]
            
    variety[i] = grape
    

variety

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\3987120411.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variety[i] = grape


0        Pinot Noir
1        Pinot Noir
2        Chardonnay
3        Pinot Noir
4        Pinot Noir
           ...     
736          Arneis
737     Pinot Blanc
738      Chardonnay
739      Pinot Gris
740    Pinot Grigio
Name: Variety, Length: 741, dtype: object

In [229]:
cleaning_df["Variety"]  = variety
cleaning_df["Variety"]

0        Pinot Noir
1        Pinot Noir
2        Chardonnay
3        Pinot Noir
4        Pinot Noir
           ...     
736          Arneis
737     Pinot Blanc
738      Chardonnay
739      Pinot Gris
740    Pinot Grigio
Name: Variety, Length: 741, dtype: object

In [185]:
## Add the styles that were found to the dataFrame
cleaning_df['style'] = None

for i, s in style:
    if len(s) == 2:
        if "Bordeaux" in s[1]:
            s = 'Bordeaux'    
        elif 'Champagne' in s[0]:
            s = 'Champagne'
            
        cleaning_df['style'][i] = s
    else:
        cleaning_df['style'][i] = s

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\3348816621.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_df['style'][i] = s
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\3348816621.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_df['style'][i] = s


In [188]:
designation = cleaning_df['Designation']
for i in designation.index:
    print(designation[i])
    print(cleaning_df['Winery'][i])
    print(cleaning_df['bottle_name'][i])
    print(cleaning_df['Variety'][i])
    print(cleaning_df['Category'][i])

    print("\n", "=================\n", "")
    

['Rascal']
The Great Oregon Wine Company
The Great Oregon Wine Company 2021 Rascal Pinot Noir (Oregon)
Pinot Noir
['Red']

 
nan
Wine by Joe
Wine by Joe 2019 Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Oregon Wine Co.']
Duck Pond Cellars
Duck Pond Cellars 2021 Oregon Wine Co. Chardonnay (Oregon)
Chardonnay
['White']

 
['Laurene']
Domaine Drouhin
Domaine Drouhin 2019 Laurene Pinot Noir (Dundee Hills)
Pinot Noir
['Red']

 
['Momtazi Vineyard']
Portlandia
Portlandia 2019 Momtazi Vineyard Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Unabridged']
Matzinger Davies
Matzinger Davies 2018 Unabridged Sauvignon Blanc (Columbia Gorge (OR))
Sauvignon Blanc
['White']

 
['Ridgecrest Vineyard']
Élevée Winegrowers
Élevée Winegrowers 2020 Ridgecrest Vineyard Riesling (Ribbon Ridge)
Riesling
['White']

 
['Holstein Vineyard Blanc de Noir']
Granville
Granville 2017 Holstein Vineyard Blanc de Noir Sparkling (Dundee Hills)
Blend
['Sparkling']

 
nan
Ashlyn
Ashlyn 2019 Pinot Noir (Willamette Vall

Blend
['Red']

 
['Bieze Vineyard']
Harper Voit
Harper Voit 2016 Bieze Vineyard Pinot Noir (Eola-Amity Hills)
Pinot Noir
['Red']

 
['Roads End']
Carlton Cellars
Carlton Cellars 2015 Roads End Pinot Noir (Yamhill-Carlton)
Pinot Noir
['Red']

 
nan
Failla
Failla 2016 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
Moffett
Moffett 2016 Pinot Noir (Dundee Hills)
Pinot Noir
['Red']

 
['Björnson Vineyard']
Failla
Failla 2016 Björnson Vineyard Gamay Noir (Eola-Amity Hills)
 Gamay
['Red']

 
['Estate Cuvée']
Troon
Troon 2016 Estate Cuvée Côt (Applegate Valley)
Blend
['Red']

 
['Estate Bottled']
Cardwell Hill
Cardwell Hill 2015 Estate Bottled Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Blue Line']
Pfeiffer
Pfeiffer 2013 Blue Line Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Firethorn Vineyard']
Adega Northwest
Adega Northwest 2015 Firethorn Vineyard Syrah (Columbia Valley (OR))
Syrah
['Red']

 
['Firethorn Vineyard Trillium Block']
Adega Northwest
Adega Northwest 2015 Fire

nan
Delfino
Delfino 2012 Zinfandel (Umpqua Valley)
Zinfandel
['Red']

 
['Looney Vineyard']
Archery Summit
Archery Summit 2012 Looney Vineyard Pinot Noir (Ribbon Ridge)
Pinot Noir
['Red']

 
['Primoris']
Ledger David
Ledger David 2013 Primoris Chenin Blanc (Rogue Valley)
Chenin Blanc
['White']

 
nan
Sokol Blosser
Sokol Blosser 2013 Chardonnay (Willamette Valley)
Chardonnay
['White']

 
["Papa Joe's Private Stash"]
Plaisance Ranch
Plaisance Ranch 2012 Papa Joe's Private Stash Tempranillo (Applegate Valley)
Tempranillo
['Red']

 
['Assemblage']
Dancin
Dancin 2013 Assemblage Chardonnay (Oregon)
Chardonnay
['White']

 
nan
Irvine Family Vineyards
Irvine Family Vineyards 2012 Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Blacktail Vineyard']
Siltstone
Siltstone 2014 Blacktail Vineyard Pinot Gris (Oregon)
Pinot Gris
['White']

 
['Guadalupe Vineyard']
Siltstone
Siltstone 2011 Guadalupe Vineyard Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Shea Vineyard']
WildAire
WildAire 2012 Shea Vineyard Pi

 
["O'Brien"]
Willamette Valley Vineyards
Willamette Valley Vineyards 2008 O'Brien Pinot Noir (Eola-Amity Hills)
Pinot Noir
['Red']

 
['Fuller']
Willamette Valley Vineyards
Willamette Valley Vineyards 2008 Fuller Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
["Founders' Reserve"]
Willamette Valley Vineyards
Willamette Valley Vineyards 2008 Founders' Reserve Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
I'M
I'M 2007 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
Kings Ridge
Kings Ridge 2009 Riesling (Oregon)
Riesling
['White']

 
nan
William Hatcher
William Hatcher 2007 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Rudolfo Vineyard']
Lumos
Lumos 2009 Rudolfo Vineyard Pinot Gris (Willamette Valley)
Pinot Gris
['White']

 
['99W']
RouteStock
RouteStock 2009 99W Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
['Melon de Bourgogne']
Macindoe
Macindoe 2009 Melon de Bourgogne Melon (Yamhill-Carlton)
Melon
['White']

 
nan
Castle Rock
Castle Rock 2009

nan
Chatter Creek
Chatter Creek 2000 Pinot Gris (Washington)
Pinot Gris
['White']

 
nan
Evesham Wood
Evesham Wood 1998 Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
["Knudsen Vyd Julia Lee's Block Blanc de Blancs"]
Argyle
Argyle 1996 Knudsen Vyd Julia Lee's Block Blanc de Blancs  (Willamette Valley)
Blend
['Sparkling']

 
['Eco Wine Cattrall Brothers']
Amity
Amity 1999 Eco Wine Cattrall Brothers Pinot Noir (Oregon)
Pinot Noir
['Red']

 
['Reserve']
Cristom
Cristom 1997 Reserve Pinot Noir (Willamette Valley)
Pinot Noir
['Red']

 
nan
Ponzi
Ponzi 1998 Arneis (Willamette Valley)
Arneis
['White']

 
nan
Tyee
Tyee 1996 Pinot Blanc (Willamette Valley)
Pinot Blanc
['White']

 
['Clos Electrique']
Cameron
Cameron 1997 Clos Electrique Chardonnay (Willamette Valley)
Chardonnay
['White']

 
nan
Torii Mor
Torii Mor 1998 Pinot Gris (Yamhill County)
Pinot Gris
['White']

 
nan
WillaKenzie Estate
WillaKenzie Estate 1997 Pinot Grigio (Willamette Valley)
Pinot Grigio
['White']

 


### Cleaning update

During my investigation into the grape Varietals, Most of the blends did not specify the veriatals, I saved the ones that did with their index. those that were specified were: 
- (486, Grenache & Pinot Noir), (232, Grenache-Syrah), (28, Rhône-style), (72, G-S-M Rhône-style), (100, Pinot Noir-Syrah), (130, Pinot Blanc-Pinot Gris), (333, Bordeaux-style), (448, Pinot-Chardonnay), (477, Bordeaux-style), (711, Syrah-Cabernet),

Looking into the review at 310 revealed that there is no vintage column. This needs extracted from the bottle name.


In [190]:
cleaning_df['vintage'] = cleaning_df['bottle_name'].str.extract(r"(\d{4})")
cleaning_df["vintage"]

0      2021
1      2019
2      2021
3      2019
4      2019
       ... 
736    1998
737    1996
738    1997
739    1998
740    1997
Name: vintage, Length: 741, dtype: object

In [230]:
designation = cleaning_df['Designation'].dropna()
for i in designation.index:
    designation[i] = designation[i][2:-2]
    
designation.value_counts()

Estate                    21
Reserve                   16
Shea Vineyard              8
Red Table Wine             5
Momtazi Vineyard           4
                          ..
Missoulan Wash Reserve     1
Hare Vineyard              1
Grenache                   1
Lazy River Vineyard        1
Clos Electrique            1
Name: Designation, Length: 316, dtype: int64

Willamette Valley
- Chehalem Mountains
- Dundee Hills
- Yamhill-Carlton
- Eola-Amity Hills
- McMinnville
- Laurelwood District
- Ribbon Ridge
- Van Duzer Corridor
- Tualatin Hills
- Mount Pisgah, Polk County
- Lower Long Tom
Rouge Valley
- Applegate Valley
Umpqua Valley
- Elkton Oregon
- Red Hill Douglas County
Columbia Gorge
Columbia Valley
Walla Walla Valley
- The Rocks District of Milton-Freewater
Snake River Valley

In [303]:
regions = cleaning_df["Appellation"].apply(lambda x: x[2:-2].split("', ' "))
not_oregon = []
ava = []
region = []
province = []
country = []
for i in regions.index:
    if len(regions[i]) != 4:
        #   # "Washington-Oregon" wines are located in Oregon and include Washington grapes in the making
        if "Washington" in regions[i]:
            province.append((i, "Oregon"))
            country.append((i,regions[i][-1]))
        if i == 333: ## Idaho wine that says US and America only
            ava.append((i, "Snake River Valley (ID)" ))
            region.append((i, "Snake River Valley (ID)" ))
            province.append((i,"Idaho"))
            country.append((i, "US"))

    else: 
        if "Oregon" not in regions[i][0]:
            ava.append((i, regions[i][0]))
            region.append((i,regions[i][1]))
            province.append((i,regions[i][2]))
            country.append((i,regions[i][3]))

        else: 
            region.append((i,regions[i][1]))
            province.append((i,regions[i][2]))
            country.append((i,regions[i][3]))

print(ava, '\n==================\n ')
print(region, '\n==================\n ')


[(3, 'Dundee Hills'), (4, 'Willamette Valley'), (5, 'Columbia Gorge (OR)'), (6, 'Ribbon Ridge'), (7, 'Dundee Hills'), (8, 'Willamette Valley'), (9, 'Willamette Valley'), (10, 'Yamhill-Carlton'), (11, 'McMinnville'), (12, 'Willamette Valley'), (13, 'Willamette Valley'), (14, 'Applegate Valley'), (15, 'Willamette Valley'), (17, 'Willamette Valley'), (18, 'Willamette Valley'), (19, 'Willamette Valley'), (22, 'Willamette Valley'), (23, 'Eola-Amity Hills'), (24, 'Eola-Amity Hills'), (25, 'Willamette Valley'), (26, 'Eola-Amity Hills'), (27, 'Willamette Valley'), (29, 'Eola-Amity Hills'), (30, 'Van Duzer Corridor'), (31, 'Willamette Valley'), (32, 'Willamette Valley'), (35, 'Dundee Hills'), (36, 'Dundee Hills'), (37, 'Willamette Valley'), (39, 'Willamette Valley'), (40, 'Umpqua Valley'), (41, 'Rogue Valley'), (42, 'Willamette Valley'), (45, 'Eola-Amity Hills'), (46, 'Willamette Valley'), (47, 'Rogue Valley'), (49, 'Willamette Valley'), (50, 'Willamette Valley'), (51, 'Willamette Valley'), (52

In [304]:
cleaning_df['AVA'] = None
cleaning_df['Region'] = None
cleaning_df['Province'] = None
cleaning_df['Country'] = None

for i, entry in ava:
    cleaning_df["AVA"][i] = entry
    
for i, entry in region:
    cleaning_df['Region'][i] = entry
    
for i, entry in province:
    cleaning_df['Province'][i] = entry
    
for i, entry in country:
    cleaning_df['Country'][i] = entry
    
    

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\3383465172.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_df["AVA"][i] = entry
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\3383465172.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_df['Region'][i] = entry
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\3383465172.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaning_df['Province'][i] = entry
C:\

In [305]:
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            739 non-null    object
 1   rating           741 non-null    int64 
 2   bottle_name      741 non-null    object
 3   region           741 non-null    object
 4   review_link      741 non-null    object
 5   source           741 non-null    object
 6   Designation      427 non-null    object
 7   Variety          741 non-null    object
 8   Appellation      741 non-null    object
 9   Winery           741 non-null    object
 10  Alcohol          531 non-null    object
 11  Bottle Size      741 non-null    object
 12  Category         741 non-null    object
 13  Date Published   741 non-null    object
 14  User Avg Rating  741 non-null    object
 15  style            29 non-null     object
 16  vintage          730 non-null    object
 17  AVA              541 non-null    ob

In [234]:
## Some of the designations are grape varietals and regions, we can remove those for they already have their own column
for i in designation.index:
    if designation[i] in cleaning_df['Variety'].unique():
        designation[i] = None
    else:
        print(designation[i])
        

Rascal
Oregon Wine Co.
Laurene
Momtazi Vineyard
Unabridged
Ridgecrest Vineyard
Holstein Vineyard Blanc de Noir
Belle Pente Vineyard
Momtazi Vineyard
Sparkling Blanc de Blancs
Terroir Series
Isla
Cristo Irresisto Oregon Pink Wine
Zéphirine
Estate
RPG Vineyard
Brut Cuvée Méthode Champenoise
Crater View Vineyard & Seven Hills Vineyard Grenache-Syrah-Mourvèdre
Flood Line
Estate Blanc de Noir
Evolution
The Eyrie South Block
Estate
Extended Tirage Brut
El Torero
Redford-Wetle Vineyard Renardière
Rosé of
Celestina Vineyard
Fern Creek Vineyard
Delaney Vineyard
Front Yard Vineyard
Vintage Brut
Hyland Vineyard
Cattrall Vineyard
Willow Creek Vineyard
Khayam's Block
AVA
Nisse+Hygge Red Table Wine
RMS Brut
Coury Old Vine Single Clone
Sorella 60th
Il Rinnegati
La Colina Vineyard
Oregon Reserve
Old Vines
Belle Pente Vineyard
Coles Valley Vineyard Blanc de Noir
Coles Valley Vineyard
La Chouette Vineyard
Chehalem Mountain Vineyard
Funky Jory
Natasha Skov
Griffin Martin
Estate Reserve Select Cluster
Old

In [307]:
cleaning_df['Designation']  = designation
cleaning_df['Designation']

0                Rascal
1                   NaN
2       Oregon Wine Co.
3               Laurene
4      Momtazi Vineyard
             ...       
736                 NaN
737                 NaN
738     Clos Electrique
739                 NaN
740                 NaN
Name: Designation, Length: 741, dtype: object

In [309]:
cleaning_df.head(10)

,price,rating,bottle_name,region,review_link,source,Designation,Variety,Appellation,Winery,...,Bottle Size,Category,Date Published,User Avg Rating,style,vintage,AVA,Region,Province,Country
0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Oregon Other,https://www.winemag.com/buying-guide/the-great...,Wine Enthusiast,Rascal,Pinot Noir,"['Oregon', ' Oregon Other', ' Oregon', ' US']",The Great Oregon Wine Company,...,750 ml,['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],None,2021,None,Oregon Other,Oregon,US
1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Oregon Other,https://www.winemag.com/buying-guide/wine-by-j...,Wine Enthusiast,NaN,Pinot Noir,"['Oregon', ' Oregon Other', ' Oregon', ' US']",Wine by Joe,...,750 ml,['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],None,2019,None,Oregon Other,Oregon,US
2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Oregon Other,https://www.winemag.com/buying-guide/duck-pond...,Wine Enthusiast,Oregon Wine Co.,Chardonnay,"['Oregon', ' Oregon Other', ' Oregon', ' US']",Duck Pond Cellars,...,750 ml,['White'],['9/1/2022'],['Not rated yet [Add Your Review]'],None,2021,None,Oregon Other,Oregon,US
3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,Laurene,Pinot Noir,"['Dundee Hills', ' Willamette Valley', ' Orego...",Domaine Drouhin,...,750 ml,['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],None,2019,Dundee Hills,Willamette Valley,Oregon,US
4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Willamette Valley,https://www.winemag.com/buying-guide/portlandi...,Wine Enthusiast,Momtazi Vineyard,Pinot Noir,"['Willamette Valley', ' Willamette Valley', ' ...",Portlandia,...,750 ml,['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],None,2019,Willamette Valley,Willamette Valley,Oregon,US
5,$33,88,Matzinger Davies 2018 Unabridged Sauvignon Bla...,Oregon Other,https://www.winemag.com/buying-guide/matzinger...,Wine Enthusiast,Unabridged,Sauvignon Blanc,"['Columbia Gorge (OR)', ' Oregon Other', ' Ore...",Matzinger Davies,...,750 ml,['White'],['7/1/2022'],['Not rated yet [Add Your Review]'],None,2018,Columbia Gorge (OR),Oregon Other,Oregon,US
6,$33,93,Élevée Winegrowers 2020 Ridgecrest Vineyard Ri...,Willamette Valley,https://www.winemag.com/buying-guide/elevee-wi...,Wine Enthusiast,Ridgecrest Vineyard,Riesling,"['Ribbon Ridge', ' Willamette Valley', ' Orego...",Élevée Winegrowers,...,750 ml,['White'],['5/1/2022'],['Not rated yet [Add Your Review]'],None,2020,Ribbon Ridge,Willamette Valley,Oregon,US
7,$90,91,Granville 2017 Holstein Vineyard Blanc de Noir...,Willamette Valley,https://www.winemag.com/buying-guide/granville...,Wine Enthusiast,Holstein Vineyard Blanc de Noir,Blend,"['Dundee Hills', ' Willamette Valley', ' Orego...",Granville,...,750 ml,['Sparkling'],['5/1/2022'],['Not rated yet [Add Your Review]'],"[Sparkling Blend, Sparkling]",2017,Dundee Hills,Willamette Valley,Oregon,US
8,$25,88,Ashlyn 2019 Pinot Noir (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/ashlyn-20...,Wine Enthusiast,NaN,Pinot Noir,"['Willamette Valley', ' Willamette Valley', ' ...",Ashlyn,...,750 ml,['Red'],['5/1/2022'],['Not rated yet [Add Your Review]'],None,2019,Willamette Valley,Willamette Valley,Oregon,US
9,$55,97,Domaine Divio 2020 Chardonnay (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,NaN,Chardonnay,"['Willamette Valley', ' Willamette Valley', ' ...",Domaine Divio,...,750 ml,['White'],['4/1/2022'],['Not rated yet [Add Your Review]'],None,2020,Willamette Valley,Willamette Valley,Oregon,US


In [325]:
category = cleaning_df['Category'].copy()
# cleaning_df['Category'] = enthusiast_df['Category'].copy()
for i in category.index:
    category[i] = category[i][2:-2]

category

0        Red
1        Red
2      White
3        Red
4        Red
       ...  
736    White
737    White
738    White
739    White
740    White
Name: Category, Length: 741, dtype: object

In [ ]:
cleaning_df['Category'] = catrgory